# Object Detection

In [1]:
!pip install ultralytics
#!pip install ultralytics opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 902.2/902.2 kB 17.0 MB/s eta 0:00:00


In [2]:
from google.colab.patches import cv2_imshow

**Object detection**

In [9]:
import os
import json
import cv2
from ultralytics import YOLO
import time
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# Load YOLOv8 model (pre-trained)
model = YOLO('yolov8n.pt')  # You can use other models like yolov8s.pt, yolov8m.pt, etc.

# Open the video file in the drive
video_path = '/content/drive/MyDrive/FYP/input_videos/sl_2.mp4'  # Ensure the correct path format
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Couldn't open the video file.")
    exit()

# Create output directory
output_dir = '/content/drive/MyDrive/FYP/output_video'
os.makedirs(output_dir, exist_ok=True)

frame_dir = f'/content/drive/MyDrive/FYP/extracted_frames/processed_frames_{timestamp}'  # Specify the subdirectory with timestamp
os.makedirs(frame_dir, exist_ok=True)

json_dir = '/content/drive/MyDrive/FYP/json_data'
os.makedirs(json_dir, exist_ok=True)

# Get video properties for output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
output_video_path = os.path.join(output_dir, f'output_video_{timestamp}.mp4')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# JSON file to save metadata
output_json_path = os.path.join(json_dir, f'detected_metadata_{timestamp}.json')
metadata = []

# Process each frame
frame_count = 0
start_time = time.time()  # Start time for FPS calculation
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame
    frame_resized = cv2.resize(frame, (frame_width , frame_height))

    # Run object detection on the resized frame
    results = model(frame_resized)

    # Get the frame with bounding boxes and labels drawn
    annotated_frame = results[0].plot()

    # Save metadata for the current frame
    detections = []
    for box in results[0].boxes:
        bbox = box.xyxy.tolist()[0]  # (x1, y1, x2, y2)
        class_id = int(box.cls)
        confidence = float(box.conf)
        detections.append({
            'class': model.names[class_id],
            'confidence': confidence,
            'box': bbox
        })

    metadata.append({
        'frame': frame_count,
        'detections': detections
    })

    # Write the annotated frame to the output video
    out.write(annotated_frame)

    # Save each annotated frame as an image file
    frame_output_path = os.path.join(frame_dir, f'frame_{frame_count:04d}.jpg')
    cv2.imwrite(frame_output_path, annotated_frame)

    # Optional: Display the annotated frame
    #cv2.imshow("YOLOv8 Object Detection - Video", annotated_frame)

    # Break the loop if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Calculate and display FPS
    frame_count += 1
    elapsed_time = time.time() - start_time
    if elapsed_time > 0:
        fps = frame_count / elapsed_time
        print(f"Processed Frame: {frame_count}, FPS: {fps:.2f}")

# Write metadata to JSON file
with open(output_json_path, 'w') as json_file:
    json.dump(metadata, json_file, indent=4)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed {frame_count} frames.")
print(f"Annotated video saved at: {output_video_path}")
print(f"Detection metadata saved at: {output_json_path}")




0: 384x640 7 persons, 1 car, 1 bus, 1 truck, 149.4ms
Speed: 3.5ms preprocess, 149.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Processed Frame: 1, FPS: 3.73

0: 384x640 7 persons, 1 car, 2 buss, 1 truck, 138.3ms
Speed: 5.1ms preprocess, 138.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Processed Frame: 2, FPS: 4.56

0: 384x640 8 persons, 2 cars, 1 bus, 2 trucks, 169.8ms
Speed: 5.2ms preprocess, 169.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Processed Frame: 3, FPS: 4.69

0: 384x640 5 persons, 1 bicycle, 2 cars, 1 motorcycle, 2 buss, 1 truck, 134.7ms
Speed: 4.5ms preprocess, 134.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Processed Frame: 4, FPS: 4.96

0: 384x640 6 persons, 1 bicycle, 2 cars, 1 motorcycle, 2 buss, 2 trucks, 135.3ms
Speed: 4.6ms preprocess, 135.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Processed Frame: 5, FPS: 5.14

0: 384x640 6 persons, 2 cars, 1

# Handle Incorrect Object Detection

***1. Prepare Dataset***

Extract frames from videos or use existing images containing the objects want to detect.

In [ ]:
import cv2
import os

# Path to your video file
video_path = '/content/drive/MyDrive/FYP/input_videos/sl_1.mp4'  # Update with your video path
output_dir = f'/content/drive/MyDrive/FYP/training_yolo8/video_frames_{timestamp}'  # Output directory for extracted frames

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(video_path)

# Frame extraction
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Save the extracted frame as an image
    frame_output_path = os.path.join(output_dir, f'frame_{frame_count:04d}.jpg')
    cv2.imwrite(frame_output_path, frame)
    frame_count += 1

cap.release()
print(f"Extracted {frame_count} frames to {output_dir}.")

Extracted 353 frames to /content/drive/MyDrive/FYP/training_yolo8/video_frames_20241221_173616.


***2. Annotations -  Label each image with bounding boxes and class names***

Save the annotations in YOLO format, which specifies the class ID and normalized coordinates of the bounding boxes.

In [ ]:
print(output_dir)

/content/drive/MyDrive/FYP/training_yolo8/video_frames_20241221_173616


In [ ]:
from ultralytics import YOLO
import os

# Load the YOLOv8 model
model = YOLO('yolov8s.pt')  # Change model type as needed


# Specify directories
# input_dir = '/content/drive/MyDrive/FYP/training_yolo8/video_frames_20241221_173616'  # Directory with images to annotate #output_dir of previous part will be input_dir here
input_dir=output_dir
output_annotation_dir = f'/content/drive/MyDrive/FYP/training_yolo8/annotations_{timestamp}'  # Directory to save annotations

# Create output directory
os.makedirs(output_annotation_dir, exist_ok=True)

# Run inference and save annotations
results = model.predict(source=input_dir, save=True, save_txt=True, project=output_annotation_dir)

# Print results summary
print("Inference complete. Annotations saved to:", output_annotation_dir)


100%|██████████| 21.5M/21.5M [00:00<00:00, 226MB/s]



image 1/353 /content/drive/MyDrive/FYP/training_yolo8/video_frames_20241221_173616/frame_0000.jpg: 512x640 9 persons, 3 cars, 1 motorcycle, 4 trucks, 582.5ms
image 2/353 /content/drive/MyDrive/FYP/training_yolo8/video_frames_20241221_173616/frame_0001.jpg: 512x640 6 persons, 4 cars, 1 motorcycle, 3 trucks, 507.9ms
image 3/353 /content/drive/MyDrive/FYP/training_yolo8/video_frames_20241221_173616/frame_0002.jpg: 512x640 6 persons, 2 cars, 1 motorcycle, 3 trucks, 512.9ms
image 4/353 /content/drive/MyDrive/FYP/training_yolo8/video_frames_20241221_173616/frame_0003.jpg: 512x640 6 persons, 2 cars, 1 motorcycle, 3 trucks, 953.8ms
image 5/353 /content/drive/MyDrive/FYP/training_yolo8/video_frames_20241221_173616/frame_0004.jpg: 512x640 6 persons, 3 cars, 1 motorcycle, 4 trucks, 795.2ms
image 6/353 /content/drive/MyDrive/FYP/training_yolo8/video_frames_20241221_173616/frame_0005.jpg: 512x640 7 persons, 3 cars, 2 motorcycles, 2 trucks, 768.8ms
image 7/353 /content/drive/MyDrive/FYP/training_yo

**Get unique class IDs**

In [ ]:
import os

# Path to your labels folder(from above annotation method)
labels_path = '/content/drive/MyDrive/FYP/training_yolo8/annotations_20241221_173616/predict/labels'

# Initialize a set to store unique class IDs
class_ids = set()

# Loop through each label file
for filename in os.listdir(labels_path):
    if filename.endswith('.txt'):
        with open(os.path.join(labels_path, filename), 'r') as file:
            for line in file:
                class_id = line.split()[0]  # Get the class ID
                class_ids.add(class_id)  # Add it to the set

# Print the unique class IDs and their count
print("Unique class IDs:", class_ids)
print("Number of classes:", len(class_ids))


Unique class IDs: {'5', '58', '26', '7', '9', '72', '0', '25', '56', '2', '1', '3'}
Number of classes: 12


**Verifying COCO Classes in YOLOv8**

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8 model
model = YOLO('yolov8n.pt')  # Replace with your specific model file if different
print(model.names)  # Dictionary of class IDs to names


{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

**Code to Save Class Names in a List**

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8 model
model = YOLO('yolov8n.pt')  # Replace with your specific model if different

# Convert IDs to integers and fetch corresponding names, saved in a list
class_names = [model.names[int(id_)] for id_ in class_ids]

# Print the list of class names
print("Class Names List:")
print(class_names)



Class Names List:
['bus', 'potted plant', 'handbag', 'truck', 'traffic light', 'refrigerator', 'person', 'umbrella', 'chair', 'car', 'bicycle', 'motorcycle']


**Map IDs to Sequential Indices**

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8 model
model = YOLO('yolov8n.pt')  # Replace with your specific model if different

# The class IDs for which you want names
# class_ids = ['5', '58', '26', '7', '9', '72', '0', '25', '56', '2', '1', '3']

# Map original IDs to sequential indices (0-11)
id_to_sequential = {original_id: i for i, original_id in enumerate(class_ids)}

# Map sequential indices to class names
sequential_names = {id_to_sequential[id_]: model.names[int(id_)] for id_ in class_ids}

# Save the mapped names in a list
mapped_names_list = [sequential_names[i] for i in range(len(class_ids))]

# Print results
print("ID to Sequential Mapping:")
print(id_to_sequential)
print("\nSequential Class Names List:")
print(mapped_names_list)


ID to Sequential Mapping:
{'5': 0, '58': 1, '26': 2, '7': 3, '9': 4, '72': 5, '0': 6, '25': 7, '56': 8, '2': 9, '1': 10, '3': 11}

Sequential Class Names List:
['bus', 'potted plant', 'handbag', 'truck', 'traffic light', 'refrigerator', 'person', 'umbrella', 'chair', 'car', 'bicycle', 'motorcycle']


***3. Change IDs in labels to sequential pattern***

In [ ]:
import os

# Directory containing annotation files.(output_annotation_dir or labels_path in previous sections)
label_dir= "/content/drive/MyDrive/FYP/training_yolo8/annotations_20241221_173616/predict/labels"  # Change this to your actual path

# Mapping of non-contiguous IDs to contiguous ones
id_mapping=id_to_sequential

# Iterate through all `.txt` files in the directory
for filename in os.listdir(label_dir):
    if filename.endswith('.txt'):  # Process only .txt files
        file_path = os.path.join(label_dir, filename)

        # Read the file contents
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Replace class IDs using the mapping
        updated_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts:  # Ensure the line is not empty
                old_id = parts[0]
                if old_id in id_mapping:
                    parts[0] = id_mapping[old_id]  # Replace with new ID
                updated_lines.append(' '.join(str(part) for part in parts))  # Ensure all elements are strings

        # Write the updated lines back to the file
        with open(file_path, 'w') as file:
            file.write('\n'.join(updated_lines))

print("Class IDs updated successfully in all label files.")



Class IDs updated successfully in all label files.


***4. Create train and val parts***

divide labels

In [ ]:
import os
import shutil
from pathlib import Path

# Define the directories
base_dir = "/content/drive/MyDrive/FYP/training_yolo8/annotations_20241221_173616/predict"
label_dir = Path(base_dir) / "labels"

# Create train and val directories
train_dir = Path(base_dir) / "train"
val_dir = Path(base_dir) / "val"

# Create the directories if they don't exist
train_dir.mkdir(parents=True, exist_ok=True)
val_dir.mkdir(parents=True, exist_ok=True)

# Get a list of all annotation files (labels)
label_files = sorted(label_dir.glob("*.txt"))  # Sort to maintain order

# Calculate the split index for 3/4 train and 1/4 validation
split_index = int(len(label_files) * 0.75)  # 75% for training

# Copy the first half to the train directory
for label_file in label_files[:split_index]:
    # Copy the label file to train/labels
    shutil.copy(label_file, train_dir)

# Copy the second half to the val directory
for label_file in label_files[split_index:]:
    # Copy the label file to val/labels
    shutil.copy(label_file, val_dir)

print(f"Copied {split_index} annotations and images to train and {len(label_files) - split_index} to val.")



Copied 264 annotations and images to train and 89 to val.


divide frames

In [ ]:
import os
import shutil
from pathlib import Path

# Define the base directory where the frames are located
base_dir = Path("/content/drive/MyDrive/FYP/training_yolo8/annotations_20241221_173616/predict")

# Create train and val directories for images
train_dir = base_dir / "train"
val_dir = base_dir / "val"

# Create directories if they don't exist
train_dir.mkdir(parents=True, exist_ok=True)
val_dir.mkdir(parents=True, exist_ok=True)

# Get a list of all image files (frames)
image_files = sorted(base_dir.glob("*.jpg"))  # Adjust the extension if necessary

# Calculate the split index for 3/4 train and 1/4 validation
split_index = int(len(label_files) * 0.75)  # 75% for training

# Copy the first half of frames to the train directory
for image_file in image_files[:split_index]:
    shutil.copy(image_file, train_dir)

# Copy the second half of frames to the val directory
for image_file in image_files[split_index:]:
    shutil.copy(image_file, val_dir)

# Print the results
print(f"Copied {split_index} frames to train and {len(image_files) - split_index} frames to val.")



Copied 264 frames to train and 89 frames to val.


# Create yaml file

In [ ]:
# Create a YAML configuration file
config_content = """
path: /content/drive/MyDrive/FYP/training_yolo8  # Path to your dataset
train: /content/drive/MyDrive/FYP/training_yolo8/annotations_20241221_173616/predict/train  # Training images
val: /content/drive/MyDrive/FYP/training_yolo8/annotations_20241221_173616/predict/val  # Validation images
nc: 12  # Number of classes

names:
  0: bus
  1: potted plant
  2: handbag
  3: truck
  4: traffic light
  5: refrigerator
  6: person
  7: umbrella
  8: chair
  9: car
  10: bicycle
  11: motorcycle
"""
# Save the content to a file
with open('/content/drive/MyDrive/FYP/training_yolo8/dataset.yaml', 'w') as f:
    f.write(config_content)

print("YAML configuration file created successfully.")


YAML configuration file created successfully.


# Fine tune model

In [ ]:
# Import the necessary library
from ultralytics import YOLO
import torch
# Load YOLOv8 model (pre-trained)
model = YOLO('models/yolov8n.pt')  # Ensure this path is correct

# Check if GPU is available
if torch.cuda.is_available():
    device = '0'  # Use the first GPU
else:
    device = 'cpu'  # Fallback to CPU if no GPU is available

# Train the model
model.train(
    data='/content/drive/MyDrive/FYP/training_yolo8/dataset.yaml',  # Path to your dataset.yaml
    epochs=50,         # Number of training epochs
    batch=16,          # Batch size
    imgsz=640,         # Image size
    device=device,     # Use GPU if available
    workers=4          # Adjust based on your CPU for data loading (optional)
)

# Save the trained model if needed
model.save('/content/drive/MyDrive/FYP/training_yolo8/trained_yolov8_model.pt')  # Save the trained model



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 96.1MB/s]


Ultralytics 8.3.53 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=models/yolov8n.pt, data=/content/drive/MyDrive/FYP/training_yolo8/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=

100%|██████████| 755k/755k [00:00<00:00, 18.2MB/s]


Overriding model.yaml nc=80 with nc=12

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

...

      50/50       2.2G      0.453      0.514     0.8087         85        640: 100%|██████████| 17/17 [00:05<00:00,  2.91it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.27it/s]

                   all         89       1266      0.834      0.549      0.608      0.488



50 epochs completed in 0.120 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.53 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.09it/s]


                   all         89       1266      0.718      0.582      0.653      0.542
                   bus         15         15      0.549     0.0667      0.141     0.0871
               handbag          1          1      0.841          1      0.995      0.995
                 truck         64         76      0.637          1       0.96      0.866
                person         89        618      0.732      0.907      0.919      0.747
              umbrella         10         10          1          0      0.311      0.301
                   car         89        180      0.752        0.7      0.748      0.519
               bicycle         11         14      0.296      0.214      0.223      0.128
            motorcycle         89        352       0.94       0.77      0.927       0.69
Speed: 0.2ms preprocess, 2.9ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/train


# Test using trained model

In [ ]:
import cv2
import json
import os
from datetime import datetime
from ultralytics import YOLO  # Ensure you have the correct YOLO library installed

# Timestamp for unique filenames
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Load your trained YOLOv8 model
model = YOLO('/content/drive/MyDrive/FYP/training_yolo8/trained_yolov8_model.pt')

# Open the video file
video_path = '/content/drive/MyDrive/FYP/input_videos/sl_1.mp4'  # Ensure the correct path format
cap = cv2.VideoCapture(video_path)

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Couldn't open the video file.")
    exit()

# Create output directories
output_dir = '/content/drive/MyDrive/FYP/trained_output_video'
os.makedirs(output_dir, exist_ok=True)

frame_dir = f'/content/drive/MyDrive/FYP/trained_extracted_frames/trained_processed_frames_{timestamp}'
os.makedirs(frame_dir, exist_ok=True)

json_dir = '/content/drive/MyDrive/FYP/trained_json_data'
os.makedirs(json_dir, exist_ok=True)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Initialize video writer
output_video_path = os.path.join(output_dir, f'trained_output_video_{timestamp}.mp4')
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# JSON file for metadata
output_json_path = os.path.join(json_dir, f'detected_trained_metadata_{timestamp}.json')
metadata = []

# Frame processing loop
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run object detection on the frame (no resizing for better accuracy)
    results = model(frame)

    # Get the annotated frame with bounding boxes and labels
    annotated_frame = results[0].plot()

    # Collect detection metadata
    detections = []
    for box in results[0].boxes:
        bbox = box.xyxy.tolist()[0]  # (x1, y1, x2, y2)
        class_id = int(box.cls)
        confidence = float(box.conf)
        detections.append({
            'class': model.names[class_id],
            'confidence': confidence,
            'box': bbox
        })

    metadata.append({
        'frame': frame_count,
        'detections': detections
    })

    # Write the annotated frame to the output video
    out.write(annotated_frame)

    # Save each annotated frame as an image file
    frame_output_path = os.path.join(frame_dir, f'frame_{frame_count:04d}.jpg')
    cv2.imwrite(frame_output_path, annotated_frame)

    # Increment frame count
    frame_count += 1

# Save detection metadata to JSON file
with open(output_json_path, 'w') as json_file:
    json.dump(metadata, json_file, indent=4)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

# Print results
print(f"Processed {frame_count} frames.")
print(f"Annotated video saved at: {output_video_path}")
print(f"Detection metadata saved at: {output_json_path}")




Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

0: 512x640 1 truck, 1 person, 416.3ms
Speed: 24.2ms preprocess, 416.3ms inference, 35.5ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 person, 223.7ms
Speed: 5.3ms preprocess, 223.7ms inference, 1.1ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 person, 220.1ms
Speed: 7.1ms preprocess, 220.1ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 person, 242.2ms
Speed: 9.4ms preprocess, 242.2ms inference, 1.1ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 person, 213.7ms
Speed: 8.9ms preprocess, 213.7ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 person, 219.7ms
Speed: 4